# Module 5: 回测评估分析（使用真实合约乘数）

本notebook展示使用真实期货合约规格进行回测分析的完整过程。

## 重要说明
- 使用真实的合约乘数（如黄金1000克/手，白银15千克/手）
- 不使用统一的乘数10
- 严格按照交易所规格计算

## 主要内容
1. 合约规格配置
2. 数据加载
3. 信号执行
4. 收益计算（真实乘数）
5. 绩效分析
6. 风险分析
7. 配对分析
8. 可视化

## 1. 环境初始化和合约规格

In [ ]:
# 环境初始化
import sys
import os
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 导入自定义模块
from lib.backtest import BacktestEngine, calculate_spread_pnl
from lib.performance import PerformanceCalculator
from lib.risk import RiskAnalyzer
from lib.pairs_analysis import PairAnalyzer
from configs.contract_specs import CONTRACT_SPECS, get_multiplier

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 环境初始化完成")

In [ ]:
# 展示合约规格
print("="*60)
print("期货合约规格表")
print("="*60)

specs_data = []
for symbol, specs in CONTRACT_SPECS.items():
    specs_data.append({
        '品种代码': symbol,
        '品种名称': specs['name'],
        '交易所': specs['exchange'],
        '合约乘数': specs['multiplier'],
        '单位': specs['unit'] + '/手',
        '最小变动': specs['tick_size'],
        '保证金率': f"{specs['margin_rate']:.0%}"
    })

specs_df = pd.DataFrame(specs_data)
specs_df

## 2. 加载数据

In [ ]:
# 加载交易信号
signals_df = pd.read_parquet('../data/signals/kalman_signals_final.parquet')
print(f"加载信号数据: {len(signals_df)} 条")
print(f"信号日期范围: {signals_df['date'].min()} 到 {signals_df['date'].max()}")
print(f"\n信号数据预览:")
signals_df.head()

In [ ]:
# 加载价格数据
price_data = {}
for symbol in CONTRACT_SPECS.keys():
    try:
        df = pd.read_parquet(f'../data/futures/{symbol}.parquet')
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        price_data[symbol] = df
        print(f"✓ {symbol} ({CONTRACT_SPECS[symbol]['name']}): {len(df)} 条价格记录")
    except Exception as e:
        print(f"× {symbol}: {e}")

print(f"\n成功加载 {len(price_data)} 个品种的价格数据")

## 3. 回测参数设置

In [ ]:
# 回测参数
INITIAL_CAPITAL = 5000000  # 500万初始资金
TRANSACTION_COST = 0.0002  # 万分之2交易成本
RISK_FREE_RATE = 0.03  # 3%年化无风险利率
MAX_STOP_LOSS = 0.10  # 10%最大止损
SLIPPAGE_TICKS = 3  # 3个tick滑点

print("回测参数设置:")
print(f"  初始资金: {INITIAL_CAPITAL:,.0f} 元")
print(f"  交易成本: {TRANSACTION_COST:.2%}")
print(f"  无风险利率: {RISK_FREE_RATE:.1%}")
print(f"  最大止损: {MAX_STOP_LOSS:.0%}")
print(f"  滑点: {SLIPPAGE_TICKS} ticks")

# 初始化回测引擎
engine = BacktestEngine(initial_capital=INITIAL_CAPITAL)
print("\n✓ 回测引擎初始化完成")

## 4. 执行信号配对

In [ ]:
# 信号统计
open_signals = signals_df[signals_df['action'] == 'open']
close_signals = signals_df[signals_df['action'] == 'close']

print(f"开仓信号: {len(open_signals)} 条")
print(f"平仓信号: {len(close_signals)} 条")

# 配对信号分布
print("\n配对信号分布:")
signal_stats = signals_df.groupby(['pair', 'action']).size().unstack(fill_value=0)
signal_stats['total'] = signal_stats.sum(axis=1)
signal_stats.sort_values('total', ascending=False)

In [ ]:
# 执行信号配对
signals = signals_df.sort_values('date').to_dict('records')
completed_trades = engine.execute_signals(signals)
print(f"✓ 配对完成: {len(completed_trades)} 笔完整交易")

# 展示前几笔配对
print("\n前5笔配对示例:")
for i, trade in enumerate(completed_trades[:5]):
    open_sig = trade['open_signal']
    close_sig = trade['close_signal']
    print(f"{i+1}. {open_sig['pair']}: {open_sig['date']} → {close_sig['date']}")

## 5. 计算交易收益（使用真实合约乘数）

In [ ]:
# 定义收益计算函数
def calculate_trade_pnl(trade, price_data):
    """使用真实合约乘数计算单笔交易收益"""
    open_signal = trade['open_signal']
    close_signal = trade['close_signal']
    pair = open_signal['pair']
    
    if '-' not in pair:
        return None
        
    y_symbol, x_symbol = pair.split('-')
    
    if y_symbol not in price_data or x_symbol not in price_data:
        return None
        
    try:
        open_date = pd.to_datetime(open_signal['date'])
        close_date = pd.to_datetime(close_signal['date'])
        
        # 获取价格
        y_open = price_data[y_symbol].loc[open_date, 'close']
        x_open = price_data[x_symbol].loc[open_date, 'close']
        y_close = price_data[y_symbol].loc[close_date, 'close']
        x_close = price_data[x_symbol].loc[close_date, 'close']
        
        # 获取仓位比例
        position_ratio = open_signal.get('position_ratio', '1:1')
        y_ratio, x_ratio = map(int, position_ratio.split(':'))
        
        # 获取真实合约乘数
        y_multiplier = get_multiplier(y_symbol)
        x_multiplier = get_multiplier(x_symbol)
        
        # 计算收益
        trade_data = {
            'position_ratio': position_ratio,
            'open_prices': {'Y': y_open, 'X': x_open},
            'close_prices': {'Y': y_close, 'X': x_close},
            'beta': open_signal.get('beta', 1.0),
            'y_symbol': y_symbol,
            'x_symbol': x_symbol
        }
        
        gross_pnl = calculate_spread_pnl(trade_data)
        
        # 计算名义价值和交易成本
        y_notional = y_open * y_ratio * y_multiplier
        x_notional = x_open * x_ratio * x_multiplier
        total_notional = y_notional + x_notional
        transaction_cost = total_notional * TRANSACTION_COST * 2
        
        return {
            'pair': pair,
            'open_date': open_date,
            'close_date': close_date,
            'holding_days': (close_date - open_date).days,
            'position_ratio': position_ratio,
            'y_symbol': y_symbol,
            'x_symbol': x_symbol,
            'y_multiplier': y_multiplier,
            'x_multiplier': x_multiplier,
            'y_open': y_open,
            'x_open': x_open,
            'y_close': y_close,
            'x_close': x_close,
            'y_price_change': y_close - y_open,
            'x_price_change': x_close - x_open,
            'y_notional': y_notional,
            'x_notional': x_notional,
            'total_notional': total_notional,
            'gross_pnl': gross_pnl,
            'transaction_cost': transaction_cost,
            'net_pnl': gross_pnl - transaction_cost
        }
        
    except Exception as e:
        return None

print("✓ 收益计算函数定义完成")

In [ ]:
# 计算所有交易收益
trades_with_pnl = []
failed_count = 0

for trade in completed_trades:
    result = calculate_trade_pnl(trade, price_data)
    if result:
        trades_with_pnl.append(result)
    else:
        failed_count += 1

print(f"✓ 成功计算 {len(trades_with_pnl)} 笔交易收益")
print(f"× 失败 {failed_count} 笔")

# 转换为DataFrame
trades_df = pd.DataFrame(trades_with_pnl)
print(f"\n交易数据列数: {len(trades_df.columns)}")
print(f"交易数据列名: {list(trades_df.columns)}")

## 6. 交易结果分析

In [ ]:
# 基础统计
total_trades = len(trades_df)
total_gross_pnl = trades_df['gross_pnl'].sum()
total_cost = trades_df['transaction_cost'].sum()
total_net_pnl = trades_df['net_pnl'].sum()
winning_trades = len(trades_df[trades_df['net_pnl'] > 0])
losing_trades = len(trades_df[trades_df['net_pnl'] <= 0])

print("="*60)
print("交易统计汇总")
print("="*60)
print(f"\n基础统计:")
print(f"  总交易次数: {total_trades}")
print(f"  盈利交易: {winning_trades} 笔 ({winning_trades/total_trades:.1%})")
print(f"  亏损交易: {losing_trades} 笔 ({losing_trades/total_trades:.1%})")
print(f"  平均持仓天数: {trades_df['holding_days'].mean():.1f} 天")

print(f"\n收益统计:")
print(f"  毛收益: {total_gross_pnl:,.2f} 元")
print(f"  交易成本: {total_cost:,.2f} 元")
print(f"  净收益: {total_net_pnl:,.2f} 元")
print(f"  平均每笔净收益: {total_net_pnl/total_trades:,.2f} 元")
print(f"  总收益率: {total_net_pnl/INITIAL_CAPITAL:.2%}")

In [ ]:
# 展示前5笔交易详情
print("前5笔交易详情:")
display_cols = ['pair', 'open_date', 'close_date', 'position_ratio', 
                'y_multiplier', 'x_multiplier', 'gross_pnl', 'net_pnl']
trades_df[display_cols].head()

## 7. 配对分析

In [ ]:
# 配对收益统计
pair_stats = trades_df.groupby('pair').agg({
    'net_pnl': ['sum', 'mean', 'count'],
    'y_multiplier': 'first',
    'x_multiplier': 'first',
    'total_notional': 'mean',
    'holding_days': 'mean'
}).round(2)

pair_stats.columns = ['总净收益', '平均净收益', '交易次数', 
                      'Y乘数', 'X乘数', '平均名义价值', '平均持仓天数']
pair_stats['胜率'] = trades_df.groupby('pair').apply(
    lambda x: (x['net_pnl'] > 0).sum() / len(x)
).round(3)
pair_stats = pair_stats.sort_values('总净收益', ascending=False)

print("="*60)
print("配对收益排名")
print("="*60)
pair_stats

In [ ]:
# Top和Bottom配对
print("Top 3 最佳配对:")
print(pair_stats.head(3)[['总净收益', '交易次数', '胜率', 'Y乘数', 'X乘数']])

print("\nBottom 3 最差配对:")
print(pair_stats.tail(3)[['总净收益', '交易次数', '胜率', 'Y乘数', 'X乘数']])

## 8. 典型交易示例

In [ ]:
# 展示不同配对的典型交易
sample_pairs = ['AG0-AU0', 'CU0-SN0', 'NI0-SF0', 'CU0-SS0']

for pair in sample_pairs:
    pair_trades = trades_df[trades_df['pair'] == pair]
    if len(pair_trades) > 0:
        # 选择净收益最高的一笔
        best_trade = pair_trades.nlargest(1, 'net_pnl').iloc[0]
        
        print(f"\n{pair} 最佳交易:")
        print(f"  日期: {best_trade['open_date'].date()} → {best_trade['close_date'].date()}")
        print(f"  持仓: {best_trade['holding_days']} 天")
        print(f"  仓位比例: {best_trade['position_ratio']}")
        print(f"  合约乘数: {best_trade['y_symbol']}={best_trade['y_multiplier']}, "
              f"{best_trade['x_symbol']}={best_trade['x_multiplier']}")
        print(f"  价格变化:")
        print(f"    {best_trade['y_symbol']}: {best_trade['y_open']:.1f} → {best_trade['y_close']:.1f} "
              f"({best_trade['y_price_change']:+.1f})")
        print(f"    {best_trade['x_symbol']}: {best_trade['x_open']:.1f} → {best_trade['x_close']:.1f} "
              f"({best_trade['x_price_change']:+.1f})")
        print(f"  名义价值: {best_trade['total_notional']:,.0f} 元")
        print(f"  毛收益: {best_trade['gross_pnl']:,.2f} 元")
        print(f"  交易成本: {best_trade['transaction_cost']:,.2f} 元")
        print(f"  净收益: {best_trade['net_pnl']:,.2f} 元")
        print(f"  收益率: {best_trade['net_pnl']/best_trade['total_notional']:.2%}")

## 9. 绩效指标计算

In [ ]:
# 计算绩效指标
calculator = PerformanceCalculator(initial_capital=INITIAL_CAPITAL, risk_free_rate=RISK_FREE_RATE)
metrics = calculator.calculate_comprehensive_metrics(trades_with_pnl)

print("="*60)
print("绩效指标")
print("="*60)
print(f"\n资金变化:")
print(f"  初始资金: {INITIAL_CAPITAL:,.0f} 元")
print(f"  最终资金: {metrics['final_capital']:,.2f} 元")
print(f"  总收益率: {metrics['total_return']:.2%}")

print(f"\n收益指标:")
print(f"  年化收益率: {metrics['annualized_return']:.2%}")
print(f"  年化波动率: {metrics['annualized_volatility']:.2%}")
print(f"  Sharpe比率: {metrics['sharpe_ratio']:.4f}")

print(f"\n风险指标:")
print(f"  最大回撤: {metrics['max_drawdown']:.2%}")
print(f"  回撤持续期: {metrics['drawdown_duration']} 个交易周期")

print(f"\n交易质量:")
print(f"  胜率: {metrics['win_rate']:.2%}")
print(f"  盈亏比: {metrics['profit_loss_ratio']:.2f}")
print(f"  平均盈利: {metrics['avg_win']:,.2f} 元")
print(f"  平均亏损: {metrics['avg_loss']:,.2f} 元")

## 10. 风险分析

In [ ]:
# 风险度量
returns = trades_df['net_pnl'].values / INITIAL_CAPITAL
analyzer = RiskAnalyzer(confidence_levels=[0.95, 0.99])
risk_metrics = analyzer.analyze_comprehensive_risk(returns, trades_with_pnl)

print("="*60)
print("风险分析")
print("="*60)

print(f"\n分布特征:")
print(f"  收益率标准差: {risk_metrics['std']:.4f}")
print(f"  偏度: {risk_metrics['skewness']:.4f}")
print(f"  峰度: {risk_metrics['kurtosis']:.4f}")

print(f"\n尾部风险:")
print(f"  VaR (95%): {risk_metrics['var_95']:.4f}")
print(f"  CVaR (95%): {risk_metrics['cvar_95']:.4f}")
print(f"  VaR (99%): {risk_metrics['var_99']:.4f}")
print(f"  CVaR (99%): {risk_metrics['cvar_99']:.4f}")

print(f"\n下行风险:")
print(f"  Sortino比率: {risk_metrics['sortino_ratio']:.4f}")

print(f"\n连续亏损:")
print(f"  最大连续亏损次数: {risk_metrics['max_losing_streak']}")
print(f"  最大连续亏损金额: {risk_metrics['max_losing_amount']:,.2f} 元")

## 11. 可视化分析

In [ ]:
# 累计收益曲线
trades_df_sorted = trades_df.sort_values('close_date').reset_index(drop=True)
trades_df_sorted['cumulative_pnl'] = trades_df_sorted['net_pnl'].cumsum()

plt.figure(figsize=(14, 6))
plt.plot(trades_df_sorted['close_date'], trades_df_sorted['cumulative_pnl'], 
         linewidth=2, label='累计净收益')
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.title('累计收益曲线（使用真实合约乘数）', fontsize=14)
plt.xlabel('日期')
plt.ylabel('累计收益（元）')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"最终累计收益: {trades_df_sorted['cumulative_pnl'].iloc[-1]:,.2f} 元")

In [ ]:
# 收益分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 净收益分布
axes[0].hist(trades_df['net_pnl'], bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='r', linestyle='--', linewidth=2, label='盈亏分界')
axes[0].axvline(x=trades_df['net_pnl'].mean(), color='g', linestyle='--', 
                linewidth=2, label=f'均值: {trades_df["net_pnl"].mean():.0f}')
axes[0].set_title('交易净收益分布', fontsize=12)
axes[0].set_xlabel('净收益（元）')
axes[0].set_ylabel('频次')
axes[0].legend()

# 持仓天数分布
axes[1].hist(trades_df['holding_days'], bins=20, edgecolor='black', alpha=0.7, color='green')
axes[1].axvline(x=trades_df['holding_days'].mean(), color='r', linestyle='--', 
                linewidth=2, label=f'均值: {trades_df["holding_days"].mean():.1f}天')
axes[1].set_title('持仓天数分布', fontsize=12)
axes[1].set_xlabel('持仓天数')
axes[1].set_ylabel('频次')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# 配对收益贡献图
pair_pnl = trades_df.groupby('pair')['net_pnl'].sum().sort_values()

plt.figure(figsize=(10, 8))
colors = ['red' if x < 0 else 'green' for x in pair_pnl.values]
pair_pnl.plot(kind='barh', color=colors)
plt.title('各配对净收益贡献（使用真实合约乘数）', fontsize=14)
plt.xlabel('净收益（元）')
plt.ylabel('配对')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(True, alpha=0.3, axis='x')

# 添加数值标签
for i, (pair, value) in enumerate(pair_pnl.items()):
    plt.text(value, i, f'{value:,.0f}', 
             ha='left' if value > 0 else 'right', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# 回撤分析
cumulative_capital = INITIAL_CAPITAL + trades_df_sorted['cumulative_pnl']
cumulative_returns = cumulative_capital / INITIAL_CAPITAL
running_max = cumulative_returns.expanding().max()
drawdown = (cumulative_returns - running_max) / running_max

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 累计收益率
axes[0].plot(trades_df_sorted['close_date'], cumulative_returns, 
             linewidth=2, label='累计收益率', color='blue')
axes[0].plot(trades_df_sorted['close_date'], running_max, 
             '--', alpha=0.5, label='历史最高', color='green')
axes[0].set_ylabel('累计收益率')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)
axes[0].set_title('累计收益率与回撤分析（使用真实合约乘数）', fontsize=14)

# 回撤
axes[1].fill_between(trades_df_sorted['close_date'], drawdown, 0, 
                     color='red', alpha=0.3, label='回撤')
axes[1].plot(trades_df_sorted['close_date'], drawdown, 
             color='red', linewidth=1)
axes[1].set_ylabel('回撤率')
axes[1].set_xlabel('日期')
axes[1].grid(True, alpha=0.3)
axes[1].legend(loc='lower left')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"最大回撤: {drawdown.min():.2%}")
max_dd_idx = drawdown.idxmin()
print(f"最大回撤发生日期: {trades_df_sorted.loc[max_dd_idx, 'close_date'].date()}")

## 12. 合约乘数影响分析

In [ ]:
# 比较真实乘数和统一乘数的差异
def calculate_uniform_pnl(row):
    """使用统一乘数10计算收益"""
    y_ratio, x_ratio = map(int, row['position_ratio'].split(':'))
    y_pnl = row['y_price_change'] * y_ratio * 10  # 统一乘数10
    x_pnl = -row['x_price_change'] * x_ratio * 10  # 做空
    return y_pnl + x_pnl

trades_df['uniform_pnl'] = trades_df.apply(calculate_uniform_pnl, axis=1)
trades_df['pnl_diff'] = trades_df['gross_pnl'] - trades_df['uniform_pnl']
trades_df['pnl_diff_pct'] = (trades_df['pnl_diff'] / trades_df['gross_pnl'].abs() * 100).round(1)

print("="*60)
print("合约乘数影响分析")
print("="*60)

# 总体影响
total_real_pnl = trades_df['gross_pnl'].sum()
total_uniform_pnl = trades_df['uniform_pnl'].sum()
print(f"\n总体影响:")
print(f"  使用真实乘数总收益: {total_real_pnl:,.2f} 元")
print(f"  使用统一乘数总收益: {total_uniform_pnl:,.2f} 元")
print(f"  差异: {total_real_pnl - total_uniform_pnl:,.2f} 元")
print(f"  差异率: {(total_real_pnl - total_uniform_pnl)/total_uniform_pnl*100:.1f}%")

# 各配对的影响
print("\n各配对乘数影响（差异最大的5个）:")
pair_impact = trades_df.groupby('pair').agg({
    'gross_pnl': 'sum',
    'uniform_pnl': 'sum',
    'pnl_diff': 'sum',
    'y_multiplier': 'first',
    'x_multiplier': 'first'
}).round(2)

pair_impact['diff_pct'] = (pair_impact['pnl_diff'] / pair_impact['uniform_pnl'].abs() * 100).round(1)
pair_impact = pair_impact.sort_values('pnl_diff', key=abs, ascending=False)

print(pair_impact.head()[['gross_pnl', 'uniform_pnl', 'pnl_diff', 'diff_pct', 'y_multiplier', 'x_multiplier']])

## 13. 总结报告

In [ ]:
# 生成总结报告
print("="*70)
print("金属期货配对交易策略回测报告（使用真实合约乘数）".center(70))
print("="*70)

print(f"\n【回测概要】")
print(f"  回测期间: {trades_df['open_date'].min().date()} 至 {trades_df['close_date'].max().date()}")
print(f"  初始资金: {INITIAL_CAPITAL:,.0f} 元")
print(f"  最终资金: {metrics['final_capital']:,.2f} 元")
print(f"  净收益: {total_net_pnl:,.2f} 元")
print(f"  收益率: {total_net_pnl/INITIAL_CAPITAL:.2%}")

print(f"\n【绩效指标】")
print(f"  年化收益率: {metrics['annualized_return']:.2%}")
print(f"  年化波动率: {metrics['annualized_volatility']:.2%}")
print(f"  Sharpe比率: {metrics['sharpe_ratio']:.4f}")
print(f"  最大回撤: {metrics['max_drawdown']:.2%}")
print(f"  Sortino比率: {risk_metrics['sortino_ratio']:.4f}")

print(f"\n【风险指标】")
print(f"  VaR (95%): {risk_metrics['var_95']:.4f}")
print(f"  CVaR (95%): {risk_metrics['cvar_95']:.4f}")
print(f"  最大连续亏损: {risk_metrics['max_losing_streak']} 次")

print(f"\n【交易统计】")
print(f"  总交易次数: {total_trades}")
print(f"  胜率: {winning_trades/total_trades:.2%}")
print(f"  盈亏比: {metrics['profit_loss_ratio']:.2f}")
print(f"  平均持仓天数: {trades_df['holding_days'].mean():.1f}")

print(f"\n【最佳配对Top 3】")
for i, (pair, stats) in enumerate(pair_stats.head(3).iterrows()):
    print(f"  {i+1}. {pair}: {stats['总净收益']:,.2f} 元 (乘数: {stats['Y乘数']:.0f} vs {stats['X乘数']:.0f})")

print(f"\n【最差配对Bottom 3】")
for i, (pair, stats) in enumerate(pair_stats.tail(3).iterrows()):
    print(f"  {i+1}. {pair}: {stats['总净收益']:,.2f} 元 (乘数: {stats['Y乘数']:.0f} vs {stats['X乘数']:.0f})")

print(f"\n【合约乘数影响】")
print(f"  使用真实乘数 vs 统一乘数(10):")
print(f"    收益差异: {total_real_pnl - total_uniform_pnl:,.2f} 元")
print(f"    相对差异: {(total_real_pnl - total_uniform_pnl)/total_uniform_pnl*100:.1f}%")

print("\n" + "="*70)
print("报告完成".center(70))
print("="*70)

## 14. 保存结果

In [ ]:
# 保存回测结果
output_dir = '../data/backtest_results'
os.makedirs(output_dir, exist_ok=True)

# 保存交易记录
trades_df.to_csv(f'{output_dir}/trades_real_multipliers_detailed.csv', index=False)
print(f"✓ 交易记录已保存至: {output_dir}/trades_real_multipliers_detailed.csv")

# 保存配对统计
pair_stats.to_csv(f'{output_dir}/pair_stats_real_multipliers.csv')
print(f"✓ 配对统计已保存至: {output_dir}/pair_stats_real_multipliers.csv")

# 保存绩效指标
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv(f'{output_dir}/performance_metrics_real.csv', index=False)
print(f"✓ 绩效指标已保存至: {output_dir}/performance_metrics_real.csv")

# 保存风险指标
risk_df = pd.DataFrame([risk_metrics])
risk_df.to_csv(f'{output_dir}/risk_metrics_real.csv', index=False)
print(f"✓ 风险指标已保存至: {output_dir}/risk_metrics_real.csv")

print(f"\n所有结果已保存至 {output_dir} 目录")